# Segmenting and clustering Neighborhoods in Toronto - Part 3

## Import the merged data from previous part

In [1]:
import pandas as pd
data_merged = pd.read_csv('segnclus_neighborhood_part2.csv')

data_merged.drop(columns='Unnamed: 0', axis=1, inplace=True)
data_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Ger the latitude and longitude value of Toronto

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: ...working... done

# All requested packages already installed.

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto with neighborgoods superimposed on top

In [3]:
import folium

In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Borough'], data_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Make data with all the Boroughs that has "Toronto" in their name

In [5]:
data_toronto = data_merged[data_merged['Borough'].str.contains("Toronto")]
data_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


## Generate a map of Toronto with the updated dataset

In [6]:
# add markers to map
for lat, lng, borough, neighborhood in zip(data_toronto['Latitude'], data_toronto['Longitude'], data_toronto['Borough'], data_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare credentials

In [7]:
CLIENT_ID = 'KUB1KGCZNZWRTEB0BFEJ0ULICH3PLZWCBHRDYMTOYYT1CR01'
CLIENT_SECRET = 'AIFQYMDGURLSXAQE0MKBVZWTJOSIGOFUSV4N5ZNX3GCGZD2Y'
ACCESS_TOKEN = '40C3QLJ5GQRWY1CB3NC2CBH1B5Z5MNWUGOLIEEFZHFTSXASL'
VERSION = '20201205'
LIMIT = 5 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)
print('Access Token: ' + ACCESS_TOKEN)

My credentails:
CLIENT_ID: KUB1KGCZNZWRTEB0BFEJ0ULICH3PLZWCBHRDYMTOYYT1CR01
CLIENT_SECRET: AIFQYMDGURLSXAQE0MKBVZWTJOSIGOFUSV4N5ZNX3GCGZD2Y
Access Token: 40C3QLJ5GQRWY1CB3NC2CBH1B5Z5MNWUGOLIEEFZHFTSXASL


## Start exploring the neighborhoods in dataset ...

### Define a function to get top 50 venues with 500 meters radius in every neighborhood present in our toronto dataset

In [8]:
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
     
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        #print(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues = getNearbyVenues(names=data_toronto['Neighbourhood'], latitudes=data_toronto['Latitude'], longitudes=data_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [11]:
print('Shape of the generated dataset is: ', toronto_venues.shape)
toronto_venues.head()

Shape of the generated dataset is:  (187, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Check some statistics about the dataset
1. Check the number of values returned for each neighborhood
2. Check how many unique categories are present in the dataset

In [12]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",5,5,5,5,5,5
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",5,5,5,5,5,5
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",5,5,5,5,5,5
Central Bay Street,5,5,5,5,5,5
Christie,5,5,5,5,5,5
Church and Wellesley,5,5,5,5,5,5
"Commerce Court, Victoria Hotel",5,5,5,5,5,5
Davisville,5,5,5,5,5,5


In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 82 uniques categories.


## Analyze each neighborhood

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,...,Salon / Barbershop,Spa,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhoods and take the mean of the frequency 

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,...,Salon / Barbershop,Spa,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.2
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
5,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
6,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.2,0.00,0.0
7,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
8,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0
9,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0


Get top 5 most common venues

In [16]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0  Vegetarian / Vegan Restaurant   0.2
1                       Beer Bar   0.2
2                   Liquor Store   0.2
3                     Restaurant   0.2
4                         Museum   0.2


----Brockton, Parkdale Village, Exhibition Place----
                       venue  freq
0                Coffee Shop   0.4
1         Italian Restaurant   0.2
2                        Gym   0.2
3                        Bar   0.2
4  Middle Eastern Restaurant   0.0


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
            venue  freq
0      Comic Shop   0.2
1     Pizza Place   0.2
2   Burrito Place   0.2
3  Farmers Market   0.2
4         Brewery   0.2


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Airport Food Court   0.2
1      Airport Lounge   0.2
2    Airport Terminal   0.2
3            

Display top 10 venues for each neighborhood

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
import numpy as np 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Vegetarian / Vegan Restaurant,Restaurant,Museum,Beer Bar,Liquor Store,Donut Shop,Dog Run,Coffee Shop,Comic Shop,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bar,Gym,Italian Restaurant,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
2,"Business reply mail Processing Centre, South C...",Comic Shop,Farmers Market,Burrito Place,Brewery,Pizza Place,Deli / Bodega,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport,Airport Food Court,Airport Lounge,Airport Terminal,Harbor / Marina,Vegetarian / Vegan Restaurant,Diner,Creperie,Cuban Restaurant,Dance Studio
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Fish & Chips Shop,Fast Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio


## Clustering of neighborhoods (with k-mean clustering)

In [24]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 1, 0, 3, 2, 2, 3, 1])

In [38]:
# add clustering labels
#print(neighborhoods_venues_sorted.columns)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data_toronto
dat = neighborhoods_venues_sorted

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(dat.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Bakery,Distribution Center,Restaurant,Spa,Coffee Shop,Donut Shop,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Yoga Studio,Italian Restaurant,Park,Creperie,Coffee Shop,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Plaza,Comic Shop,Burrito Place,Clothing Store,Eastern European Restaurant,Dance Studio,Fish & Chips Shop,Concert Hall,Fast Food Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Restaurant,Gastropub,Japanese Restaurant,Creperie,Coffee Shop,Department Store,Concert Hall,Cosmetics Shop,Cuban Restaurant,Dance Studio
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Trail,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store


## Point the clusters on map

In [42]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Yoga Studio,Italian Restaurant,Park,Creperie,Coffee Shop,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop
15,Downtown Toronto,0,Restaurant,Gastropub,Japanese Restaurant,Creperie,Coffee Shop,Department Store,Concert Hall,Cosmetics Shop,Cuban Restaurant,Dance Studio
24,Downtown Toronto,0,Coffee Shop,Middle Eastern Restaurant,Fish & Chips Shop,Fast Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
43,West Toronto,0,Coffee Shop,Bar,Gym,Italian Restaurant,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
54,East Toronto,0,Coffee Shop,Fish Market,Pet Store,Ice Cream Shop,Bookstore,Vegetarian / Vegan Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


### Cluster 2

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,1,Health Food Store,Pub,Trail,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store
31,West Toronto,1,Middle Eastern Restaurant,Grocery Store,Bakery,Bar,Brewery,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
36,Downtown Toronto,1,Deli / Bodega,Lake,Park,Dessert Shop,Vegetarian / Vegan Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
61,Central Toronto,1,Bus Line,Swim School,Park,Lawyer,Vegetarian / Vegan Restaurant,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop,Creperie
67,Central Toronto,1,Food & Drink Shop,Hotel,Park,Breakfast Spot,Department Store,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop
68,Central Toronto,1,Trail,Jewelry Store,Sushi Restaurant,Park,Vegetarian / Vegan Restaurant,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop,Creperie
69,West Toronto,1,Flea Market,Gastropub,Italian Restaurant,Park,Bar,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
74,Central Toronto,1,Café,Burger Joint,Donut Shop,Park,Indian Restaurant,Deli / Bodega,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
75,West Toronto,1,Dessert Shop,Movie Theater,Eastern European Restaurant,Dog Run,Cuban Restaurant,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop,Creperie
80,Downtown Toronto,1,Beer Bar,Japanese Restaurant,Italian Restaurant,Dessert Shop,Restaurant,Vegetarian / Vegan Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


### Cluster 3

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Bakery,Distribution Center,Restaurant,Spa,Coffee Shop,Donut Shop,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop
20,Downtown Toronto,2,Vegetarian / Vegan Restaurant,Restaurant,Museum,Beer Bar,Liquor Store,Donut Shop,Dog Run,Coffee Shop,Comic Shop,Concert Hall
30,Downtown Toronto,2,Vegetarian / Vegan Restaurant,Plaza,Concert Hall,Hotel,Restaurant,Comic Shop,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
42,Downtown Toronto,2,Pub,Gym,Bakery,Restaurant,Coffee Shop,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
48,Downtown Toronto,2,Pub,Gym,Museum,Restaurant,Café,Dog Run,Distribution Center,Coffee Shop,Comic Shop,Concert Hall
73,Central Toronto,2,Yoga Studio,Spa,Restaurant,Salon / Barbershop,Mexican Restaurant,Airport Terminal,Clothing Store,Comic Shop,Concert Hall,Cosmetics Shop
83,Central Toronto,2,Playground,Tennis Court,Park,Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie
86,Central Toronto,2,Liquor Store,Sushi Restaurant,Supermarket,American Restaurant,Restaurant,Deli / Bodega,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop
92,Downtown Toronto,2,Vegetarian / Vegan Restaurant,Thai Restaurant,Museum,Park,Restaurant,Dance Studio,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop
97,Downtown Toronto,2,Restaurant,Gym,Gym / Fitness Center,Bakery,Vegetarian / Vegan Restaurant,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


### Cluster 4

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,3,Café,Plaza,Comic Shop,Burrito Place,Clothing Store,Eastern European Restaurant,Dance Studio,Fish & Chips Shop,Concert Hall,Fast Food Restaurant
25,Downtown Toronto,3,Café,Grocery Store,Italian Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Deli / Bodega,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
79,Central Toronto,3,Dessert Shop,Café,Pizza Place,Indian Restaurant,Deli / Bodega,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
81,West Toronto,3,Café,Fish & Chips Shop,Burrito Place,Sushi Restaurant,Coffee Shop,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
84,Downtown Toronto,3,Café,Organic Grocery,Arts & Crafts Store,Bakery,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio


### Cluster 5

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,West Toronto,4,Cuban Restaurant,Brewery,Asian Restaurant,Pizza Place,Ice Cream Shop,Vegetarian / Vegan Restaurant,Deli / Bodega,Concert Hall,Cosmetics Shop,Creperie
41,East Toronto,4,Ice Cream Shop,Cosmetics Shop,Greek Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Department Store,Concert Hall,Creperie,Cuban Restaurant,Dance Studio
47,East Toronto,4,Fish & Chips Shop,Fast Food Restaurant,Sushi Restaurant,Gym,Ice Cream Shop,Department Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
62,Central Toronto,4,Home Service,Garden,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store
